<a href="https://colab.research.google.com/github/Ry02024/SIGNATE/blob/main/%E5%82%B5%E5%8B%99%E4%B8%8D%E5%B1%A5%E8%A1%8C/%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%E3%83%A2%E3%83%87%E3%83%AB%E3%81%AE%E7%94%9F%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#予測モデル生成

In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [ ]:
class ModelTrainingAndEvaluation:
    def __init__(self, train_df, test_df):
        self.train_df = train_df
        self.test_df = test_df
        self.le_exps = {}  # カテゴリカル変数用のエンコーダーを保存する辞書
        self.le_target = LabelEncoder()  # 目的変数用のエンコーダー
        self.model = None
        self.metrics = None
        # トレーニングデータとテストデータの差分から目的変数を特定
        self.target_col_name = list(set(train_df.columns) - set(test_df.columns))[0]
        # 分割されたデータセットを保持するための属性
        self.X_train = None
        self.y_train = None
        self.y_train_encoded = None
        self.eval_metrics = self.lgb_f1_score # 評価関数をインスタンス属性に割り当て

    def __call__(self):
        # 前処理と訓練を行う
        self.preprocess_and_train()
        # テストデータセットの準備
        # self.prepare_test_dataset()

    # 評価指標としてF1スコアを使用するカスタム関数
    @staticmethod
    def lgb_f1_score(y_hat, data):
        y_true = data.get_label()
        y_hat = np.round(y_hat)  # 予測値の四捨五入
        return 'f1', f1_score(y_true, y_hat), True

    def preprocess_and_train(self):
        # カテゴリカルデータと数値データを分ける
        categorical_cols = self.train_df.select_dtypes(include=['object', 'category']).columns.drop(self.target_col_name, errors='ignore').tolist()
        numerical_cols = self.train_df.select_dtypes(include=[np.number]).columns.drop(self.target_col_name, errors='ignore').tolist()
        if self.target_col_name in numerical_cols:
            numerical_cols.remove(self.target_col_name)

        # 目的変数をエンコード
        self.le_target
        self.train_df[self.target_col_name] = self.le_target.fit_transform(self.train_df[self.target_col_name])

        self.le_exps = {}
        # カテゴリカルデータをラベルエンコーディング
        for col in categorical_cols:
            le = LabelEncoder()
            all_data = pd.concat([self.train_df[col], self.test_df[col]], axis=0).astype(str)
            le.fit(all_data)
            self.train_df[col] = le.transform(self.train_df[col].astype(str))
            self.test_df[col] = le.transform(self.test_df[col].astype(str))
            self.le_exps[col] = le

        # 目的変数と特徴量を分割
        self.X_train = self.train_df.drop(columns=[self.target_col_name])
        self.y_train = self.train_df[self.target_col_name]

        # y_trainに対してラベルエンコーディングを適用する場合
        self.y_train_encoded = self.le_target.fit_transform(self.y_train.astype(str))

        # モデルの訓練
        # パラメータ設定
        params = {
            'objective': 'binary',
            'metric': 'custom',
            'learning_rate': 0.1,
        }
        # LightGBMのデータセット形式に変換
        lgb_train = lgb.Dataset(self.X_train, self.y_train_encoded, categorical_feature=categorical_cols, free_raw_data=False)

        # モデルの訓練
        self.model = lgb.train(params, lgb_train, feval=self.eval_metrics, num_boost_round=100)

    def prepare_test_dataset(self):
        # テストデータセットの特徴量とラベルを準備し、目的変数をエンコードする
        self.X_test = self.test_df
        # self.y_test_encoded = self.le_target.transform(self.test_df[self.target_col_name])

    def evaluate_model(self):
        # テストデータセットで評価を行う
        y_pred = np.round(self.model.predict(self.X_train))
        self.metrics = {
            'accuracy': accuracy_score(self.y_train, y_pred),
            'precision': precision_score(self.y_train, y_pred),
            'recall': recall_score(self.y_train, y_pred),
            'f1_score': f1_score(self.y_train, y_pred),
            'roc_auc': roc_auc_score(self.y_train, self.model.predict(self.X_train))
        }
        return self.metrics

In [ ]:
train4pred = train.copy()
test4pred = test.copy()

In [ ]:
# 使用例
model_trainer = ModelTrainingAndEvaluation(train4pred, test4pred)
model_trainer()  # モデルの訓練とテストデータセットの準備を行う
metrics = model_trainer.evaluate_model()  # 訓練済みモデルの評価を行う
print(metrics)

In [ ]:
# 仮定: `lgb_model`は既に訓練されたLightGBMモデルのインスタンスです。
# モデルをファイルに保存
model_trainer.model.save_model('model.txt')

In [ ]:
model_trainer.model.params

In [ ]:
model_trainer.model.predict(model_trainer.X_train)

#データ

In [ ]:
#kaggle.jsonをGoogle Drive からひっぱってくるやつ
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'signate.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/root/.signate/signate.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [ ]:
!pip install -q signate

  Preparing metadata (setup.py) ... done


In [ ]:
!mkdir data

In [ ]:
!signate download --competition-id=266

sample_submit.csv

test.csv

train.csv


Download completed.


#AutoGluo

In [ ]:
!pip install autogluon -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.1/229.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.0/306.0 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.7/416.7 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.5/120.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 90.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:
!pip install --upgrade autogluon

In [ ]:
from autogluon.tabular import TabularPredictor
import pandas as pd

In [ ]:
# 仮想のデータファイルパス
train_data_path = 'train.csv'  # 訓練データファイルのパス
test_data_path = 'test.csv'  # テストデータファイルのパス

# データの読み込み
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

# 目的変数の列名（ここでは例として 'target' を使用）
label_column = list(set(train_data.columns) - set(test_data.columns))[0]

In [ ]:
from autogluon.tabular import TabularPredictor
import pandas as pd

# TabularPredictorの初期化
predictor = TabularPredictor(label=label_column, eval_metric='mean_squared_error')

# ハイパーパラメータの調整範囲を指定
hyperparameters = {
    'GBM': [
        {'num_boost_round': 100, 'num_leaves': 31},  # デフォルトの設定
        {'num_boost_round': 100, 'num_leaves': 64},  # より多くの葉を持つモデル
    ],
    #'NN': [{}],  # デフォルトのニューラルネットワーク
    'CAT': [{}],  # デフォルトのCatBoostモデル
}

# 早期停止の設定
time_limit = 1800  # 1時間で学習を停止

# モデルの訓練
predictor.fit(train_data=train_data, hyperparameters=hyperparameters, time_limit=time_limit, presets='optimize_for_deployment')

No path specified. Models will be saved in: "AutogluonModels/ag-20240305_094253"
Presets specified: ['optimize_for_deployment']
Beginning AutoGluon training ... Time limit = 1800s
AutoGluon will save models to "AutogluonModels/ag-20240305_094253"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sat Nov 18 15:31:17 UTC 2023
CPU Count:          8
Memory Avail:       46.27 GB / 50.99 GB (90.7%)
Disk Space Avail:   166.58 GB / 201.23 GB (82.8%)
Train Data Rows:    55583
Train Data Columns: 28
Label Column:       y
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and label-values can't be converted to int).
	Label info (max, min, mean, stddev): (1998.9999999999995, 1.0, 160.16365, 168.08714)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter d

In [ ]:
ls

In [ ]:
pd.read_csv('train.csv', header=None)

In [ ]:
predictor = TabularPredictor(label=label_column, problem_type='regression').fit(train_data=train_data)

In [ ]:
# テストデータから目的変数を除外（もし存在する場合）
test_data_nolabel = test_data.drop(columns=[label_column])
# 予測の実行
predictions = predictor.predict(test_data_nolabel)

# 真の値がテストデータに含まれている場合、性能を評価する
# ここでは、真の値がテストデータに含まれていると仮定しています
performance = predictor.evaluate(test_data)

In [ ]:
# 予測結果を表示または保存
print(predictions)
# predictions.to_csv('predictions.csv', index=False)